# Création d'un classificateur multi-label pour étiqueter les montres.

Un réseau de neurones ne se limite pas à modéliser la distribution d'une seule variable. En fait, il peut facilement gérer les cas où chaque image est associée à plusieurs étiquettes. Dans cette recette, nous allons implémenter un CNN pour classer le genre et le style/l'usage des montres. Commençons.

nous utiliserons l'ensemble de données Fashion Product Images (Small) hébergé dans Kaggle, que, après vous être connecté, vous pouvez télécharger ici : https://www.kaggle.com/paramaggarwal/fashion-product-images-small .

In [1]:
!pip install -q kaggle

Vous devrez télécharger votre fichier kaggle.json pour cette étape, qui peut être obtenu à partir de votre compte Kaggle

In [3]:
from google.colab import files

files.upload()

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [6]:
#!!kaggle datasets download -d paramaggarwal/fashion-product-images-small

In [8]:
#!unzip fashion-product-images-small.zip

Passons en revue les étapes pour terminer la recette:

**1.** Importez les packages nécessaires :

In [9]:
import os
import pathlib
from csv import DictReader

import glob
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import *

** 2.** Dénir une fonction pour construire l'architecture du réseau. Tout d'abord, implémentez les blocs convolutifs :

In [10]:
def build_network(width, height, depth, classes):
    input_layer = Input(shape=(width, height, depth))

    x = Conv2D(filters=32,
               kernel_size=(3, 3),
               padding='same')(input_layer)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters=32,
               kernel_size=(3, 3),
               padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(rate=0.25)(x)

    x = Conv2D(filters=64,
               kernel_size=(3, 3),
               padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters=64,
               kernel_size=(3, 3),
               padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(rate=0.25)(x)

    x = Flatten()(x)
    x = Dense(units=512)(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Dropout(rate=0.25)(x)

    x = Dense(units=classes)(x)
    output = Activation('sigmoid')(x)

    return Model(input_layer, output)

**3.** Dénissez une fonction pour charger toutes les images et les étiquettes (genre et usage), à ​​partir d'une liste de chemins d'images et d'un dictionnaire de métadonnées associé à chacune d'entre elles :

In [18]:
def load_images_and_labels(image_paths, styles, target_size):
    images = []
    labels = []

    for image_path in image_paths:
        image = load_img(image_path, target_size=target_size)
        image = img_to_array(image)
        image_id = image_path.split(os.path.sep)[-1][:-4]

        image_style = styles[image_id]
        label = (image_style['gender'], image_style['usage'])

        images.append(image)
        labels.append(label)

    return np.array(images), np.array(labels)

**4.** Définissez la graine aléatoire pour garantir la reproductibilité :

In [11]:
SEED = 999
np.random.seed(SEED)

**5.** Dénissez les chemins d'accès aux images et au fichier de métadonnées styles.csv

In [13]:
base_path = pathlib.Path('/content/myntradataset')
styles_path = str(base_path / 'styles.csv')
images_path_pattern = str(base_path / 'images/*.jpg')
image_paths = glob.glob(images_path_pattern)

**6.** Conservez uniquement les images des montres pour une utilisation décontractée, décontractée et formelle, adaptée aux hommes et aux femmes :

In [16]:
with open(styles_path, 'r') as f:
    dict_reader = DictReader(f)
    STYLES = [*dict_reader]

    article_type = 'Watches'
    genders = {'Men', 'Women'}
    usages = {'Casual', 'Smart Casual', 'Formal'}
    STYLES = {style['id']: style
              for style in STYLES
              if (style['articleType'] == article_type and
                  style['gender'] in genders and
                  style['usage'] in usages)}

image_paths = [*filter(lambda p: p.split(os.path.sep)[-1][:-4]
                                 in STYLES.keys(),
                       image_paths)]

**7.** Chargez les images et les étiquettes, en redimensionnant les images dans une forme 64x64x3 :


In [19]:
X, y = load_images_and_labels(image_paths, STYLES, (64, 64))
X = X.astype('float') / 255.0


**8.** Normalisez les images et encodez les étiquettes en multi-hot :

In [21]:
#X = X.astype('float') / 255.0
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y)

**9.** Créez les fractionnements d'entraînement, de validation et de test :

In [23]:
(X_train, X_test,
 y_train, y_test) = train_test_split(X, y,
                                     stratify=y,
                                     test_size=0.2,
                                     random_state=SEED)
(X_train, X_valid,
 y_train, y_valid) = train_test_split(X_train, y_train,
                                      stratify=y_train,
                                      test_size=0.2,
                                      random_state=SEED)

**10.** Construisez et compilez le réseau :

In [24]:
model = build_network(width=64,
                      height=64,
                      depth=3,
                      classes=len(mlb.classes_))
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

**11.** Entraînez le modèle pour 20 époques, par lots de 64 images à la fois :

In [25]:
BATCH_SIZE = 64
EPOCHS = 20
model.fit(X_train, y_train,
          validation_data=(X_valid, y_valid),
          batch_size=BATCH_SIZE,
          epochs=EPOCHS)

Epoch 1/20
24/24 [==============================] - 33s 97ms/step - loss: 0.6418 - accuracy: 0.1679 - val_loss: 0.9724 - val_accuracy: 0.0027
Epoch 2/20
24/24 [==============================] - 1s 59ms/step - loss: 0.4181 - accuracy: 0.2893 - val_loss: 0.7068 - val_accuracy: 0.4058
Epoch 3/20
24/24 [==============================] - 1s 60ms/step - loss: 0.2923 - accuracy: 0.4267 - val_loss: 0.6802 - val_accuracy: 0.6393
Epoch 4/20
24/24 [==============================] - 1s 59ms/step - loss: 0.2120 - accuracy: 0.5627 - val_loss: 0.4499 - val_accuracy: 0.9814
Epoch 5/20
24/24 [==============================] - 1s 61ms/step - loss: 0.1642 - accuracy: 0.6549 - val_loss: 0.5956 - val_accuracy: 0.9814
Epoch 6/20
24/24 [==============================] - 1s 62ms/step - loss: 0.1327 - accuracy: 0.6835 - val_loss: 1.8023 - val_accuracy: 0.9735
Epoch 7/20
24/24 [==============================] - 1s 60ms/step - loss: 0.1085 - accuracy: 0.6808 - val_loss: 0.4444 - val_accuracy: 0.9814
Epoch 8/20
2

**12.** Évaluez le modèle sur l'ensemble de test :

In [26]:
result = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print(f'Test accuracy: {result[1]}')

8/8 [==============================] - 0s 47ms/step - loss: 0.3343 - accuracy: 0.9130
Test accuracy: 0.9129511713981628


**13.** Utilisez le modèle pour faire des prédictions sur une image de test, affichant la probabilité de chaque étiquette :

In [27]:
test_image = np.expand_dims(X_test[0], axis=0)
probabilities = model.predict(test_image)[0]
for label, p in zip(mlb.classes_, probabilities):
    print(f'{label}: {p * 100:.2f}%')

Casual: 99.92%
Formal: 0.00%
Men: 53.19%
Smart Casual: 0.00%
Women: 63.96%


**14.** Comparez les étiquettes de vérité avec la prédiction du réseau :

In [29]:
ground_truth_labels = np.expand_dims(y_test[0], axis=0)
ground_truth_labels = mlb.inverse_transform(ground_truth_labels)
print(f'Ground truth labels: {ground_truth_labels}')

Ground truth labels: [('Casual', 'Women')]


Nous avons implémenté une version plus petite d'un réseau VGG, capable d'effectuer une classification multi-étiquettes et multi-classes, en modélisant des distributions indépendantes pour les métadonnées de genre et d'utilisation associées à chaque montre. En d'autres termes, nous avons modélisé deux problèmes de classification binaire en même temps : un pour le genre et un pour l'usage. C'est la raison pour laquelle nous avons activé les sorties du réseau avec Sigmoid, au lieu de Sofmax, et aussi pourquoi la fonction de perte utilisée est binary_crossentropy et non catégorique_crossentropy.

Nous avons entraîné le réseau susmentionné sur 20 époques, sur des lots de 64 images à la fois, obtenant une précision respectable de 90 % sur l'ensemble de test. Enfin, nous avons fait une prédiction sur une image invisible du jeu de test et vérifié que les étiquettes produites avec une grande certitude par le réseau (100 % de certitude pour Casual et 99,16 % pour Women) correspondent aux catégories de vérité  Casual et Women .